In [2]:
library(DiffBind)
library(tidyverse)

In [3]:
wkdir <- '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/3_ChIP/4_DiffBind'
setwd(wkdir)

In [4]:
samples.sheet <- '2017_PNAS_Li_simplified.csv'

In [16]:
dbObj <- dba(sampleSheet=samples.sheet)

STAT5B-noIL2-CD8_1 Spleen STAT5B_noIL2 noIL2 invitro 1 bed

STAT5B-IL2-CD8_1 Spleen STAT5B_IL2 IL2 invitro 1 bed



In [17]:
dbObj <- dba.count(dbObj, bUseSummarizeOverlaps=TRUE)

Warning message:
“fragmentSize ignored when bUseSummarizeOverlaps is TRUE in dba.count”


In [18]:
dbObj

2 Samples, 356 sites in matrix:
                  ID Tissue       Factor Condition Treatment Replicate Caller
1 STAT5B-noIL2-CD8_1 Spleen STAT5B_noIL2     noIL2   invitro         1 counts
2   STAT5B-IL2-CD8_1 Spleen   STAT5B_IL2       IL2   invitro         1 counts
  Intervals FRiP
1       356 0.01
2       356 0.01

In [20]:
dbObj <- dba.contrast(dbObj, categories=DBA_FACTOR, minMembers=1)

ERROR: Error in dba.contrast(dbObj, categories = DBA_FACTOR, minMembers = 1): minMembers must be at least 2. Use of replicates strongly advised.
